# load the AGORA2 models and media

In [1]:
from pandas import read_excel

nutrients_tbl = read_excel("nutrient list_20231219.xlsx")
display(nutrients_tbl)

,name,CAS,SMILES
0,adrenic acid,28874-58-0,CCCCC/C=C\C/C=C\C/C=C\C/C=C\CCCCCC(=O)O
1,acetic acid,64-19-7,CC(=O)O
2,alanine,56-41-7,C[C@@H](C(=O)O)N
3,alcohol,64-17-5,CCO
4,alpha-carotene,7488-99-5,CC1=C(C(CCC1)(C)C)/C=C/C(=C/C=C/C(=C/C=C/C=C(\...
...,...,...,...
115,vitamin d3,67-97-0,C[C@H](CCCC(C)C)[C@H]1CC[C@@H]\2[C@@]1(CCC/C2=...
116,vitamin k,12001-79-5,CC1=C(C(=O)C2=CC=CC=C2C1=O)CC=C(C)CCCC(C)CCCC(...
117,xylitol,87-99-0,C(C(C(C(CO)O)O)O)O
118,xylose,10257-31-5,C1[C@H]([C@@H]([C@H](C(O1)O)O)O)O


In [2]:
from modelseedpy.biochem import from_local

msdb = from_local("../../ModelSEEDDatabase/")
print(dir(msdb.compounds[0]))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_annotation', '_bound', '_id', '_model', '_reaction', '_repr_html_', '_set_id_with_model', 'abbr', 'annotation', 'charge', 'compartment', 'constraint', 'copy', 'delta_g', 'delta_g_error', 'elements', 'flags', 'formula', 'formula_weight', 'id', 'inchi', 'inchi_key', 'is_abstract', 'is_cofactor', 'is_core', 'is_obsolete', 'linked_compound', 'mass', 'model', 'name', 'names', 'notes', 'pka', 'pkb', 'reactions', 'remove_from_model', 'seed_id', 'shadow_price', 'smiles', 'source', 'summary', 'to_template_compartment_compound', 'y']


In [3]:
from cobra.io import read_sbml_model
from glob import glob

def get_model(path):
    try:  model = read_sbml_model(path)
    except:  return

# models = []
# for model in glob("../../AGORA2_models/*.xml"):
#     try:     models.append(read_sbml_model(model))
#     except:  print(model, " is broken")

from multiprocess import Pool
pool = Pool(24)
args = glob("../../AGORA2_models/*.xml")
print(args[0])
pool.map(get_model, args)

../../AGORA2_models/Klebsiella_aerogenes_CAV1320.xml


# simulate the community

In [ ]:
%run ../../ModelSEEDpy/modelseedpy/community/MSCommunity.py
